In [1]:
# CMPT459 2020 spring
# group name: Salt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline
from sklearn import model_selection
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RepeatedKFold 
from sklearn import tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_score, recall_score

def outlier_fd(data, threshold=0.5):
    low, high = np.percentile(data,1), np.percentile(data, 99)
    dist = high - low
    cut_off = dist * threshold
    lower, upper = low-cut_off, high+cut_off
    return [lower, upper]

def photos_number_counter(x):
    return len(x)

def valid_des(s):
    valid_s = " "
    for i in s.split():
        if i.isalnum():
            valid_s = " ".join([valid_s, i])
            
    return valid_s.strip()

In [2]:
def dataforclf(train):
    #Find the outlier of price, latitude and longitude using bound given by outlier_fd
    [price_lower, price_higher] = outlier_fd(train['price'])
    outlier_price = train[(train['price']<price_lower)|(train['price']>price_higher)]
    [latitude_lower, latitude_higher] = outlier_fd(train['latitude'],1)
    outlier_latitude = train[(train['latitude']<latitude_lower)|(train['latitude']>latitude_higher)]
    [longitude_lower, longitude_higher] = outlier_fd(train['longitude'],1)
    outlier_longitude = train[(train['longitude']<longitude_lower)|(train['longitude']>longitude_higher)]
    #deal with outliers, set the column for modified data and plot again
    train['price_modify'] = train['price']
    train['price_modify'].loc[train['price_modify']<price_lower] = price_lower
    train['price_modify'].loc[train['price_modify']>price_higher] = price_higher
    #deal with outliers, set the column for modified data and plot again
    train['latitude_modify'] = train['latitude']
    train['latitude_modify'].loc[train['latitude_modify']<latitude_lower] = latitude_lower
    train['latitude_modify'].loc[train['latitude_modify']>latitude_higher] = latitude_higher
    #deal with outliers, set the column for modified data and plot again
    train['longitude_modify'] = train['longitude']
    train['longitude_modify'].loc[train['longitude']<longitude_lower] = longitude_lower
    train['longitude_modify'].loc[train['longitude']>longitude_higher] = longitude_higher
    #get the number of photos
    train['photos_number'] = train['photos'].apply(photos_number_counter)
    #set all feature words in tf_idf as features for train for feature_modify
    train['features_modify'] = train['features'].apply(lambda x:" ".join(["_".join(i.split(" "))for i in x]))
    tfidf_f = TfidfVectorizer(stop_words=['central_a', 'pre', 'war','features'], max_features=98)
    train_sparse_f = tfidf_f.fit_transform(train['features_modify'])
    feature_list = tfidf_f.get_feature_names()
    array1 = train_sparse_f.toarray()
    for i, feature_str in enumerate(feature_list):
        list_all = []
        [rows, cols] = array1.shape
        for row in range(rows):
            list_all.append(array1[row][i])
        feature_str = feature_str + '(f)'
        feature_v = pd.Series(list_all,train.index, name = feature_str)
        train[feature_str] = feature_v
    #set all feature words in tf_idf as features for train for description_modify
    train['description_modify'] = train['description'].apply(lambda x:valid_des(x))
    tfidf_d = TfidfVectorizer(stop_words=['features'], max_features=100)
    train_sparse_d = tfidf_d.fit_transform(train['description_modify'])
    description_list = tfidf_d.get_feature_names()
    array2 = train_sparse_d.toarray()
    for i, description_str in enumerate(description_list):
        list_all = []
        [rows, cols] = array2.shape
        for row in range(rows):
            list_all.append(array2[row][i])
        description_str_ = description_str + '(d)'
        description_v = pd.Series(list_all,train.index, name = description_str_)
        train[description_str_] = description_v
    ## /*add something to change the created to year month day*/
    train['created']=pd.to_datetime(train["created"])
    #extract pieces
    train['year']=train['created'].dt.year
    train['month']=train['created'].dt.month
    train['day']=train['created'].dt.day
    ## /*add something to do feature selection*/
    train = train.drop(["building_id", "created", "description", "display_address", "features", 
                         "manager_id", "photos", "street_address", "price", "longitude",
                        "latitude", "features_modify", "description_modify"], axis=1)
    #drop stop words
    train = train.drop(['an(d)','and(d)','are(d)','as(d)','at(d)','be(d)','by(d)','can(d)','for(d)','from(d)','in(d)','is(d)','it(d)','just(d)','me(d)','more(d)','natural(d)','new(d)','or(d)','park(d)','that(d)','the(d)','this(d)','to(d)','will(d)','with(d)','you(d)','your(d)'], axis = 1)
    return train

In [3]:
#main function
train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [4]:
train['test']=False
test['test']=True

cdf = train.append(test).reset_index(drop=True)
cdf = dataforclf(cdf)
train = cdf[cdf.test == False]
test = cdf[cdf.test == True]
train = train.drop(["test"],axis=1)
test = test.drop(["interest_level","test"],axis=1)

E:\Anaconda\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
E:\Anaconda\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
test

,bathrooms,bedrooms,listing_id,price_modify,latitude_modify,longitude_modify,photos_number,_photos(f),actual_apt(f),balcony(f),...,supports(d),text(d),two(d),unit(d),west(d),windows(d),york(d),year,month,day
49352,1.0,1,7142618,2950.0,40.7185,-73.9865,8,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,11
49353,1.0,2,7210040,2850.0,40.7278,-74.0000,3,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,24
49354,1.0,0,7174566,2295.0,40.7260,-74.0026,1,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,17
49355,1.0,2,7191391,2900.0,40.7321,-74.0028,4,0.0,0.0,0.0,...,0.330474,0.000000,0.0,0.000000,0.0,0.0,0.292075,2016,6,21
49356,1.0,1,7171695,3254.0,40.7054,-74.0095,6,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124006,1.0,1,6928108,1700.0,40.7925,-73.9454,10,0.0,0.0,0.0,...,0.000000,0.273799,0.0,0.000000,0.0,0.0,0.000000,2016,4,26
124007,1.0,2,6906674,4195.0,40.7456,-73.9797,4,0.0,0.0,0.0,...,0.000000,0.188096,0.0,0.000000,0.0,0.0,0.000000,2016,4,21
124008,1.0,0,6897967,2400.0,40.7416,-73.9829,0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.135853,0.0,0.0,0.000000,2016,4,20
124009,2.0,2,6842183,6895.0,40.7485,-73.9800,8,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.193046,2016,4,8


In [6]:
train

,bathrooms,bedrooms,interest_level,listing_id,price_modify,latitude_modify,longitude_modify,photos_number,_photos(f),actual_apt(f),...,supports(d),text(d),two(d),unit(d),west(d),windows(d),york(d),year,month,day
0,1.0,1,medium,7170325,2400.0,40.7108,-73.9539,12,0.0,0.0,...,0.000000,0.277420,0.0,0.000000,0.0,0.0,0.000000,2016,6,16
1,1.0,2,low,7092344,3800.0,40.7513,-73.9722,6,0.0,0.0,...,0.000000,0.158305,0.0,0.000000,0.0,0.0,0.000000,2016,6,1
2,1.0,2,medium,7158677,3495.0,40.7575,-73.9625,6,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,14
3,1.5,3,medium,7211212,3000.0,40.7145,-73.9425,5,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,24
4,1.0,0,low,7225292,2795.0,40.7439,-73.9743,4,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,6,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49347,1.0,3,low,6824800,2800.0,40.8433,-73.9396,5,0.0,0.0,...,0.125354,0.000000,0.0,0.000000,0.0,0.0,0.110788,2016,4,5
49348,1.0,2,medium,6813268,2395.0,40.8198,-73.9578,5,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.228953,2016,4,2
49349,1.0,1,medium,6927093,1850.0,40.5765,-73.9554,3,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,2016,4,26
49350,1.0,2,medium,6892816,4195.0,40.7448,-74.0017,5,0.0,0.0,...,0.000000,0.000000,0.0,0.163351,0.0,0.0,0.000000,2016,4,19


In [7]:
#For training
x_train = train.drop(["listing_id","interest_level","photos_number","day"],axis=1)
#For testing
x_test = test.drop(["listing_id","photos_number","day"],axis=1)

y = pd.get_dummies(train['interest_level'])
#X_train,X_test,Y_train,Y_test = train_test_split(x_train,y,test_size=0.33,random_state=0)

# Frist Version under final choice of features

In [8]:
Best_validation_score = 0
kf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=None)
best_loss=999
for train_index, test_index in kf.split(x_train):
    x_train['id'] = list(range(0,49352))
    x_train.set_index("id", inplace=True)
    y['id'] = list(range(0,49352))
    y.set_index("id", inplace=True)
    X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
    Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
    Tree = OneVsRestClassifier(tree.DecisionTreeClassifier())
    Tree.fit(X_train,Y_train)
    print("Current training accuracy: %0.5f" % Tree.score(X_train,Y_train))
    Validation_score = Tree.score(X_test,Y_test)
    if Validation_score > Best_validation_score:
        Best_validation_score = Validation_score
        Best_Tree = Tree
    Y_labeled = Tree.predict_proba(X_test)
    loss = log_loss(Y_test,Y_labeled)
    if best_loss > loss:
        best_loss = loss
    print ("Current validation accuracy: %0.5f" % Validation_score)
print ("Best validation accuracy: %0.5f" % Best_validation_score)
print ("Best log loss: %0.5f" % best_loss)

Current training accuracy: 0.97334
Current validation accuracy: 0.53241
Current training accuracy: 0.97373
Current validation accuracy: 0.54113
Current training accuracy: 0.97278
Current validation accuracy: 0.55218
Current training accuracy: 0.97341
Current validation accuracy: 0.53212
Current training accuracy: 0.97442
Current validation accuracy: 0.53171
Current training accuracy: 0.97382
Current validation accuracy: 0.53556
Current training accuracy: 0.97438
Current validation accuracy: 0.54225
Current training accuracy: 0.97249
Current validation accuracy: 0.54853
Current training accuracy: 0.97321
Current validation accuracy: 0.55502
Current training accuracy: 0.97384
Current validation accuracy: 0.53698
Current training accuracy: 0.97267
Current validation accuracy: 0.53890
Current training accuracy: 0.97339
Current validation accuracy: 0.52938
Current training accuracy: 0.97373
Current validation accuracy: 0.54610
Current training accuracy: 0.97283
Current validation accuracy: 

In [9]:
Y_Prediction = Best_Tree.predict_proba(x_test)
test['listing_id'] = test['listing_id'].astype(int)
ids = test['listing_id']
labels = pd.DataFrame(Y_Prediction)
labels.reset_index(drop=True, inplace=True)
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, labels], axis=1, ignore_index=True)
out.columns = ['listing_id','high','low','medium']
out

,listing_id,high,low,medium
0,7142618,0.0,1.0,0.0
1,7210040,1.0,1.0,0.0
2,7174566,0.0,1.0,0.0
3,7191391,0.0,0.0,0.0
4,7171695,0.0,1.0,0.0
...,...,...,...,...
74654,6928108,0.0,0.0,0.0
74655,6906674,0.0,0.0,1.0
74656,6897967,0.0,1.0,0.0
74657,6842183,0.0,1.0,0.0


In [10]:
out.to_csv('submission.csv',index = False)

# Improvement on parameters of the decision tree and k-fold cross validation

In [8]:
#Choose the best k for k-fold cross validation.
best_loss = 999
for i in range(2,11):
    kf = RepeatedKFold(n_splits=i, n_repeats=1, random_state=None)
    for train_index, test_index in kf.split(x_train):
        x_train['id'] = list(range(0,49352))
        x_train.set_index("id", inplace=True)
        y['id'] = list(range(0,49352))
        y.set_index("id", inplace=True)
        X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
        Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
        Tree = OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=4,random_state=0))
        Tree.fit(X_train,Y_train)
        Y_labeled = Tree.predict_proba(X_test)
        loss = log_loss(Y_test,Y_labeled)
        if best_loss > loss:
            best_loss = loss
            res = i
print ("Best result: %d" % res)

Best result: 10


In [8]:
#Choose the best max_depth.
best_loss=999
kf = RepeatedKFold(n_splits=8, n_repeats=1, random_state=None)
for train_index, test_index in kf.split(x_train):
    x_train['id']=list(range(0,49352))
    x_train.set_index("id", inplace=True)
    y['id']=list(range(0,49352))
    y.set_index("id", inplace=True)
    X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
    Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
    for i in range(2,15):
        Tree=OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=i,random_state=0))
        Tree.fit(X_train,Y_train)
        #validation_score = Tree.score(X_test,Y_test)
        Y_labeled=Tree.predict_proba(X_test)
        loss=log_loss(Y_test,Y_labeled)
        if best_loss>loss:
            best_loss=loss
            best_depth = i
print ("Best result: %d" % best_depth)

Best result: 6


In [8]:
#Choose the best max_leaf_nodes.
best_loss = 999
kf = RepeatedKFold(n_splits=8, n_repeats=1, random_state=None)
for i in range (15,30):
    for train_index, test_index in kf.split(x_train):
        x_train['id'] = list(range(0,49352))
        x_train.set_index("id", inplace=True)
        y['id'] = list(range(0,49352))
        y.set_index("id", inplace=True)
        X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
        Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
        Tree = OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=5,random_state=0,max_leaf_nodes=i))
        Tree.fit(X_train,Y_train)
        #validation_score = Tree.score(X_test,Y_test)
        Y_labeled = Tree.predict_proba(X_test)
        loss = log_loss(Y_test,Y_labeled)
        if best_loss > loss:
            best_loss = loss
            best_res = i
print ("Best result: %d" % best_res)

Best result: 23


In [8]:
#Choose the best min_samples_leaf.
best_loss=999
kf = RepeatedKFold(n_splits=8, n_repeats=1, random_state=None)
for i in range (2,20):
    for train_index, test_index in kf.split(x_train):
        x_train['id']=list(range(0,49352))
        x_train.set_index("id", inplace=True)
        y['id']=list(range(0,49352))
        y.set_index("id", inplace=True)
        X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
        Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
        Tree=OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=5,random_state=0,max_leaf_nodes=23,min_samples_leaf=i))
        Tree.fit(X_train,Y_train)
        #validation_score = Tree.score(X_test,Y_test)
        Y_labeled=Tree.predict_proba(X_test)
        loss=log_loss(Y_test,Y_labeled)
        if best_loss>loss:
            best_loss=loss
            best_res = i
print ("Best result: %d" % best_res)

Best result: 6


In [13]:
#Choose the best criterion. The result shows the two criterion has similar effects.
kf = RepeatedKFold(n_splits=8, n_repeats=1, random_state=None)
for train_index, test_index in kf.split(x_train):
    x_train['id'] = list(range(0,49352))
    x_train.set_index("id", inplace=True)
    y['id'] = list(range(0,49352))
    y.set_index("id", inplace=True)
    X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
    Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
    Tree = OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='entropy',max_depth=6,random_state=0)).fit(X_train,Y_train)
    validation_score1 = Tree.score(X_test,Y_test)
    Y_labeled = Tree.predict_proba(X_test)
    loss1 = log_loss(Y_test,Y_labeled)
    Tree = OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=6,random_state=0)).fit(X_train,Y_train)
    validation_score2 = Tree.score(X_test,Y_test)
    Y_labeled = Tree.predict_proba(X_test)
    loss2 = log_loss(Y_test,Y_labeled)
    if validation_score1 >= validation_score2:
        print('Validation accuray: Entropy')
    else:
        print('Validation accuracy: Gini index')
    if loss1 <= loss2:
        print('Log loss: Entropy')
    else:
        print('Log loss: Gini index')


Validation accuracy: Gini index
Log loss: Gini index
Validation accuray: Entropy
Log loss: Gini index
Validation accuracy: Gini index
Log loss: Entropy
Validation accuracy: Gini index
Log loss: Gini index
Validation accuracy: Gini index
Log loss: Gini index
Validation accuray: Entropy
Log loss: Gini index
Validation accuracy: Gini index
Log loss: Gini index
Validation accuray: Entropy
Log loss: Gini index


# Final version

In [8]:
best_loss = 999
Best_validation_score = 0
kf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=None)
best_precision=0
precision=[]
recall=[]
for train_index, test_index in kf.split(x_train):
    x_train['id'] = list(range(0,49352))
    x_train.set_index("id", inplace=True)
    y['id'] = list(range(0,49352))
    y.set_index("id", inplace=True)
    X_train, X_test = x_train.loc[train_index,:], x_train.loc[test_index,:]
    Y_train, Y_test = y.loc[train_index,:], y.loc[test_index,:]
    Tree = OneVsRestClassifier(tree.DecisionTreeClassifier(criterion='gini',max_depth=6,random_state=0,splitter='best',max_leaf_nodes=23,min_samples_leaf=6))
    Tree.fit(X_train,Y_train)
    print("Current training accuracy: %0.5f" % Tree.score(X_train,Y_train))
    #accuracy
    Validation_score = Tree.score(X_test,Y_test)
    if Validation_score > Best_validation_score:
        Best_validation_score = Validation_score
        #Best_Tree = Tree   
    #log loss
    Y_labeled = Tree.predict_proba(X_test)
    loss = log_loss(Y_test,Y_labeled)
    if best_loss > loss:
        best_loss = loss
        #Best_Tree = Tree  
    #precision
    Y_Pred = Tree.predict(X_test)
    print(multilabel_confusion_matrix(Y_test, Y_Pred))
    if best_precision < precision_score(Y_test, Y_Pred, average='macro'):
        best_precision = precision_score(Y_test, Y_Pred, average='macro')
        Best_Tree = Tree  
    precision.append(precision_score(Y_test, Y_Pred, average='macro'))
    recall.append(recall_score(Y_test, Y_Pred, average='macro'))
    print ("Current validation accuracy: %0.5f" % Validation_score)
print ("Best validation accuracy: %0.5f" % Best_validation_score)
print ("Best log loss: %0.5f" % best_loss)

#Y_pred = Best_Tree.predict(X_test)
#multilabel_confusion_matrix(Y_test, Y_pred)   
print(precision)
print(recall)


Current training accuracy: 0.64387
[[[4528   23]
  [ 362   23]]

 [[ 519 1013]
  [ 314 3090]]

 [[3773   16]
  [1130   17]]]
Current validation accuracy: 0.63331
Current training accuracy: 0.64409
[[[4524   42]
  [ 319   51]]

 [[ 478  983]
  [ 315 3160]]

 [[3825   20]
  [1076   15]]]
Current validation accuracy: 0.65032
Current training accuracy: 0.60398
[[[4523   20]
  [ 370   22]]

 [[ 646  899]
  [ 511 2879]]

 [[3782    0]
  [1153    0]]]
Current validation accuracy: 0.58744


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Current training accuracy: 0.61141
[[[4534   32]
  [ 325   44]]

 [[ 621  896]
  [ 429 2989]]

 [[3785    2]
  [1148    0]]]
Current validation accuracy: 0.61398
Current training accuracy: 0.64786
[[[4552   19]
  [ 340   24]]

 [[ 439 1016]
  [ 269 3211]]

 [[3843    1]
  [1090    1]]]
Current validation accuracy: 0.65471
Current training accuracy: 0.63088
[[[4521   21]
  [ 362   31]]

 [[ 517  989]
  [ 366 3063]]

 [[3792   30]
  [1095   18]]]
Current validation accuracy: 0.62533
Current training accuracy: 0.63350
[[[4516   23]
  [ 351   45]]

 [[ 583  973]
  [ 327 3052]]

 [[3775    0]
  [1160    0]]]
Current validation accuracy: 0.62756


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Current training accuracy: 0.61031
[[[4511   32]
  [ 343   49]]

 [[ 594  881]
  [ 476 2984]]

 [[3852    0]
  [1083    0]]]
Current validation accuracy: 0.61378


...

E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Current training accuracy: 0.65088
[[[4535   36]
  [ 309   55]]

 [[ 452 1045]
  [ 267 3171]]

 [[3802    0]
  [1133    0]]]
Current validation accuracy: 0.65289


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Current training accuracy: 0.64646
[[[4503   26]
  [ 356   50]]

 [[ 474 1030]
  [ 306 3125]]

 [[3819   18]
  [1084   14]]]
Current validation accuracy: 0.64316
Current training accuracy: 0.63021
[[[4526   26]
  [ 360   23]]

 [[ 557  962]
  [ 362 3054]]

 [[3799    0]
  [1136    0]]]
Current validation accuracy: 0.62310
Best validation accuracy: 0.66687
Best log loss: 0.66905
[0.5894196658271725, 0.5798969483069364, 0.4286176443402489, 0.4494389125968074, 0.6059266472638978, 0.5756922823803376, 0.47334185848252347, 0.45899839756387123, 0.42093651215693223, 0.420654806648292, 0.5140098015586084, 0.4426381461675579, 0.5393772455499226, 0.6026167953947178, 0.4437980645033086, 0.5880977840041993, 0.40647934899431903, 0.5280728430111186, 0.6278460184140661, 0.6298755285030897, 0.4264979288660431, 0.43671075493626815, 0.6137964448008554, 0.46373877464264845, 0.4382981727423439, 0.46194323814537, 0.5417434079639004, 0.5243248194848583, 0.6160901662732878, 0.560595513234439, 0.62550575230311

E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
Y_Prediction=Best_Tree.predict_proba(x_test)
test['listing_id'] = test['listing_id'].astype(int)
ids = test['listing_id']
labels = pd.DataFrame(Y_Prediction)
labels.reset_index(drop=True, inplace=True)
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, labels], axis=1, ignore_index=True)
out.columns=['listing_id','high','low','medium']
out

,listing_id,high,low,medium
0,7142618,0.019887,0.848758,0.234660
1,7210040,0.165278,0.482436,0.226322
2,7174566,0.054809,0.666523,0.190476
3,7191391,0.165278,0.482436,0.504732
4,7171695,0.019887,0.848758,0.224175
...,...,...,...,...
74654,6928108,0.413793,0.304584,0.290961
74655,6906674,0.105691,0.561784,0.274501
74656,6897967,0.054809,0.666523,0.190476
74657,6842183,0.049593,0.877449,0.043655


In [10]:
out.to_csv('submission.csv',index = False)

In [11]:
out['listing_id'].dtype

dtype('int32')